In [0]:
import random
import numpy as np
import os
import torch 
import matplotlib.pyplot as plt

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

random.seed(13)
np.random.seed(13)
torch.manual_seed(13)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
os.environ['PYTHONHASHSEED'] = str(13)

In [0]:
!pip install transformers

### Loading texts, split on train, val and test


In [0]:
!wget https://www.dropbox.com/s/n8nfxxh5azocrrm/train_LASER.csv?dl=0
!wget https://www.dropbox.com/s/v7msk8zxkkiqc6q/X_test_translated.txt?dl=0
!wget https://www.dropbox.com/s/je9am5c77ytfcaf/test_LASER.csv?dl=0

--2020-06-08 10:52:08--  https://www.dropbox.com/s/n8nfxxh5azocrrm/train_LASER.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.1, 2620:100:6018:1::a27d:301
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/n8nfxxh5azocrrm/train_LASER.csv [following]
--2020-06-08 10:52:08--  https://www.dropbox.com/s/raw/n8nfxxh5azocrrm/train_LASER.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucaa90e67616b8ce7f2f1f0a7f9b.dl.dropboxusercontent.com/cd/0/inline/A5RhoD_Yb4QqHOhJHFsyAXdXRj8cT_XeBnKP1QG9cqpOimOujVhAWtAxWsGNZ1vds0x_RqpY-1xlJRm6QiYx8nYFjzVCVolptz0BHZx2IITXc4sdqZcc16vC45D0k0aQ1m4/file# [following]
--2020-06-08 10:52:09--  https://ucaa90e67616b8ce7f2f1f0a7f9b.dl.dropboxusercontent.com/cd/0/inline/A5RhoD_Yb4QqHOhJHFsyAXdXRj8cT_XeBnKP1QG9cqpOimOujVhAWtAxWsGNZ1vds0x_RqpY-1xlJRm6QiYx8nYFjzVCVolptz0B

In [0]:
#открываем файл с train dataset
import pandas as pd
df_train = pd.read_csv("train_LASER.csv?dl=0").dropna()
train_texts = list(df_train['abstracts'])
train_labels = list(df_train['labels'])

len(train_texts)

80948

In [0]:
#открываем файл с test_dataset
df_test = pd.read_csv("test_LASER.csv?dl=0").dropna()
test_texts = list(df_test['lemm_abstracts'])              #лемм там нет, просто название неправильное не дала
test_labels = list(df_test['labels'])

len(test_texts)

10150

In [0]:
def cleaning(text):
    clean_txt = []
    for sentence in text.split('.'):
        sentence = re.sub(r'[•⋅−]', ' ', sentence)
        pattern = re.compile(r'(\s,){2,}')                 # регулярка для того чтобы несколько подряд запятых заменять на одну
        sentence = re.sub(pattern, ', ', sentence)
        sent = sentence.strip()
        sent = re.sub(r'( )+', ' ', sent)
        sent = re.sub(' ,', ',', sent)
        if len(sent.lstrip(',').rstrip(',').strip().split()) > 2 and sent != 'ф лы, ил': 
             clean_txt.append(sent.lstrip(',').rstrip(',').strip())
    return '. '.join(clean_txt)         

In [0]:
from tqdm import tqdm_notebook as tqdm
import re
test_texts = [cleaning(text) for text in test_texts]
train_texts = [cleaning(text) for text in train_texts]

In [0]:
with open('X_test_translated.txt?dl=0', encoding='utf-8') as of:
    translated_texts = of.readlines()
    of.close()
    
translated_texts = [cleaning(text.strip()) for text in translated_texts]

In [0]:
train_labels = [label[0] for label in train_labels]
test_labels = [label[0] for label in test_labels]
train_labels[:10]

['G', 'H', 'B', 'H', 'A', 'F', 'G', 'B', 'E', 'B']

In [0]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(train_labels)
#add_labels = label_encoder.fit_transform(df_add['us_code'])
y_train[:10]

array([6, 7, 1, 7, 0, 5, 6, 1, 4, 1])

In [0]:
from collections import Counter
Counter(y_train)

Counter({0: 10556,
         1: 17284,
         2: 7792,
         3: 1368,
         4: 3764,
         5: 11810,
         6: 13760,
         7: 14614})

In [0]:
y_test = label_encoder.transform(test_labels)

In [0]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [0]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [0]:
import numpy as np
sentences_train = np.array(train_texts)
labels_train = y_train
sentences_train.shape

(80948,)

In [0]:
import numpy as np
#sentences_test = np.array(test_texts)
sentences_test = np.array(translated_texts)
labels_test = y_test
sentences_test.shape

(10150,)

In [0]:
from transformers import XLMRobertaTokenizer
# Load the BERT tokenizer.
print('Loading XLM-R tokenizer...')
tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base', do_lower_case=True)
#tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Loading XLM-R tokenizer...


In [0]:
# Print the original sentence.
print(' Original: ', sentences_train[0])

# Print the sentence split into tokens.
print('Tokenized: ', tokenizer.tokenize(sentences_train[0]))

# Print the sentence mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentences_train[0])))

 Original:  a wireless type prepaid ic card system enhances the convenience of users and assures the safe payment of tolls by vehicles including bikes when introduced for toll roads. this toll collecting system is composed of a card processor that is arranged in a booth for the toll collection process and has an antenna unit for executing the card process through the wireless communication with ic cards, antenna units arranged at the left and right roadsides of the booth for executing the card process through the wireless communication with ic cards, a vehicle class discriminator for discriminating classes of vehicles entering into a traffic lane, and a traffic lane controller that selects an antenna unit to perform the card process according to a discriminated vehicle class and controls the card process
Tokenized:  ['▁a', '▁wireless', '▁type', '▁pre', 'paid', '▁', 'ic', '▁card', '▁system', '▁enhance', 's', '▁the', '▁conveni', 'ence', '▁of', '▁users', '▁and', '▁assure', 's', '▁the', '▁

In [0]:
# Tokenize all of the sentences and map the tokens to thier word IDs.

input_ids_train = []

# For every sentence...
for sent in tqdm(sentences_train):
    # `encode` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    encoded_sent = tokenizer.encode(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'

                        # This function also supports truncation and conversion
                        # to pytorch tensors, but we need to do padding, so we
                        # can't use these features :( .
                        max_length = 256         # Truncate all sentences.
                        #return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.
    input_ids_train.append(encoded_sent)

# Print sentence 0, now as a list of IDs.
print('Original: ', sentences_train[0])
print('Token IDs:', input_ids_train[0])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  



Original:  a wireless type prepaid ic card system enhances the convenience of users and assures the safe payment of tolls by vehicles including bikes when introduced for toll roads. this toll collecting system is composed of a card processor that is arranged in a booth for the toll collection process and has an antenna unit for executing the card process through the wireless communication with ic cards, antenna units arranged at the left and right roadsides of the booth for executing the card process through the wireless communication with ic cards, a vehicle class discriminator for discriminating classes of vehicles entering into a traffic lane, and a traffic lane controller that selects an antenna unit to perform the card process according to a discriminated vehicle class and controls the card process
Token IDs: [0, 10, 135051, 10644, 479, 138925, 6, 1771, 20596, 5426, 156856, 7, 70, 50080, 6620, 111, 72095, 136, 92784, 7, 70, 46002, 81997, 111, 43584, 7, 390, 117001, 26719, 15528, 

In [0]:
# We'll borrow the `pad_sequences` utility function to do this.
from keras.preprocessing.sequence import pad_sequences

MAX_LEN = 256

print('\nPadding/truncating all sentences to %d values...' % MAX_LEN)

print('\nPadding token: "{:}", ID: {:}'.format(tokenizer.pad_token, tokenizer.pad_token_id))

# Pad our input tokens with value 0.
# "post" indicates that we want to pad and truncate at the end of the sequence,
# as opposed to the beginning.
input_ids_train = pad_sequences(input_ids_train, maxlen=MAX_LEN, dtype="long", 
                          value=1, truncating="post", padding="post")
#input_ids_test = pad_sequences(input_ids_test, maxlen=MAX_LEN, dtype="long", 
        #                  value=0, truncating="post", padding="post")

print('\nDone.')

Using TensorFlow backend.



Padding/truncating all sentences to 256 values...

Padding token: "<pad>", ID: 1

Done.


In [0]:
# Create attention masks
attention_masks_tr = []

# For each sentence...
for sent in tqdm(input_ids_train):
    
    # Create the attention mask.
    #   - If a token ID is 0, then it's padding, set the mask to 0.
    #   - If a token ID is > 0, then it's a real token, set the mask to 1.
    att_mask = [int(token_id != 1) for token_id in sent]
    
    # Store the attention mask for this sentence.
    attention_masks_tr.append(att_mask)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


In [0]:
# Use train_test_split to split our data into train and validation sets for
# training
from sklearn.model_selection import train_test_split

# Use 85% for training and 15% for validation.
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids_train, labels_train, 
                                                            random_state=42, test_size=0.15, shuffle=True, stratify=y_train)

# Do the same for the masks.
train_masks, validation_masks, _, _ = train_test_split(attention_masks_tr, labels_train,
                                             random_state=42, test_size=0.15, shuffle=True, stratify=y_train)

In [0]:
len(train_inputs)

68805

In [0]:
# Convert all inputs and labels into torch tensors, the required datatype 
# for our model.
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)

train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)

train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

In [0]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

batch_size = 8

# Create the DataLoader for our training set.
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# Create the DataLoader for our validation set.
validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)


In [0]:
from transformers import XLMRobertaForSequenceClassification, AdamW

# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model = XLMRobertaForSequenceClassification.from_pretrained(
    "xlm-roberta-base", # Use the 12-layer BERT model, with an uncased vocab. #bert-base-multiligual-cased
    num_labels = 8, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

# Tell pytorch to run this model on the GPU.
model.cuda()

XLMRobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): Layer

In [0]:
model.roberta.embeddings.word_embeddings.requires_grad = False
model.roberta.embeddings.word_embeddings.requires_grad

False

In [0]:
from transformers import XLMRobertaModel, AdamW

# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
xlmr = XLMRobertaModel.from_pretrained(
    "xlm-roberta-base")

In [0]:
class LSTM(nn.Module):
    def __init__(self, hidden_dim, dropout):
        super().__init__()
        self.xlmr = xlmr
        self.lstm = nn.LSTM(768, hidden_dim, bidirectional=True, batch_first=True)
        self.linear = nn.Linear(2*hidden_dim, 225)
        self.dropout = nn.Dropout(dropout)
            
    def forward(self, input_ids, attention_mask):
        xlmr_output = self.dropout(self.xlmr(input_ids, attention_mask)[0])
        output, (hidden, _) = self.lstm(xlmr_output)
      #  print(hidden.shape)
        final = torch.cat([hidden[0], hidden[1]], axis=1)
        preds = self.linear(final).squeeze(0)
        return F.log_softmax(preds, dim=1)

In [0]:
model = LSTM(hidden_dim=512, dropout = 0.2).cuda()

In [0]:
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8, # args.adam_epsilon  - default is 1e-8.
               weight_decay=1e-3)

#loss_func = nn.NLLLoss()

In [0]:
#веса для лосс функции
distrib_class = dict(Counter(y_train))
values_class = np.zeros((len(distrib_class)))
for key in sorted(distrib_class.keys()):
    values_class[key] = distrib_class[key]
    
weights = torch.FloatTensor(np.min(values_class)/values_class).cuda()

loss_func = nn.NLLLoss(weight = weights)

In [0]:
from transformers import get_linear_schedule_with_warmup

# Number of training epochs (authors recommend between 2 and 4)
epochs = 3

# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [0]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [0]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))


In [0]:
# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128


# Store the average loss after each epoch so we can plot them.
loss_values = []
loss_history = []
 
# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

    #    outputs = model(b_input_ids, 
     #               attention_mask=b_input_mask)
        outputs = model(b_input_ids, 
                    token_type_ids=None, 
                    attention_mask=b_input_mask, 
                    labels=b_labels)
        
        # The call to `model` always returns a tuple, so we need to pull the 
        # loss value out of the tuple.
        loss = outputs[0]
     #   loss = loss_func(outputs, b_labels)
        loss_history.append(loss)

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over the training data.
    avg_train_loss = total_loss / len(train_dataloader)            
    
    # Store the loss value for plotting the learning curve.
    loss_values.append(avg_train_loss)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(format_time(time.time() - t0)))
    
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        
        # Telling the model not to compute or store gradients, saving memory and
        # speeding up validation
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # This will return the logits rather than the loss because we have
            # not provided labels.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
       #     logits = model(b_input_ids, 
        #                    attention_mask=b_input_mask)
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # Get the "logits" output by the model. The "logits" are the output
        # values prior to applying an activation function like the softmax.

        # Move logits and labels to CPU

        logits = outputs[0]
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # Calculate the accuracy for this batch of test sentences.
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        # Accumulate the total accuracy.
        eval_accuracy += tmp_eval_accuracy

        # Track the number of batches
        nb_eval_steps += 1

    # Report the final accuracy for this validation run.
    print("  Accuracy: {0:.4f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!") 


======== Epoch 1 / 3 ========
Training...
  Batch    40  of  8,601.    Elapsed: 0:00:10.
  Batch    80  of  8,601.    Elapsed: 0:00:21.
  Batch   120  of  8,601.    Elapsed: 0:00:31.
  Batch   160  of  8,601.    Elapsed: 0:00:41.
  Batch   200  of  8,601.    Elapsed: 0:00:51.
  Batch   240  of  8,601.    Elapsed: 0:01:01.
  Batch   280  of  8,601.    Elapsed: 0:01:11.
  Batch   320  of  8,601.    Elapsed: 0:01:21.
  Batch   360  of  8,601.    Elapsed: 0:01:31.
  Batch   400  of  8,601.    Elapsed: 0:01:41.
  Batch   440  of  8,601.    Elapsed: 0:01:51.
  Batch   480  of  8,601.    Elapsed: 0:02:01.
  Batch   520  of  8,601.    Elapsed: 0:02:11.
  Batch   560  of  8,601.    Elapsed: 0:02:21.
  Batch   600  of  8,601.    Elapsed: 0:02:32.
  Batch   640  of  8,601.    Elapsed: 0:02:42.
  Batch   680  of  8,601.    Elapsed: 0:02:52.
  Batch   720  of  8,601.    Elapsed: 0:03:02.
  Batch   760  of  8,601.    Elapsed: 0:03:12.
  Batch   800  of  8,601.    Elapsed: 0:03:22.
  Batch   840  of

In [0]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []

# For every sentence...
for sent in tqdm(sentences_test):
    # `encode` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    encoded_sent = tokenizer.encode(
                        sent, max_length = 256,                    # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                   )
    
    input_ids.append(encoded_sent)

# Pad our input tokens
input_ids = pad_sequences(input_ids, maxlen=256, value=1,
                          dtype="long", truncating="post", padding="post")

# Create attention masks
attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i!=1) for i in seq]
  attention_masks.append(seq_mask) 

# Convert to tensors.
prediction_inputs = torch.tensor(input_ids)
prediction_masks = torch.tensor(attention_masks)
prediction_labels = torch.tensor(y_test)

# Set the batch size.  
batch_size = 32  

# Create the DataLoader.
prediction_data = TensorDataset(prediction_inputs, prediction_masks, prediction_labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


In [0]:
print('Predicting labels for {:,} test sentences...'.format(len(prediction_inputs)))

# Put model in evaluation mode
model.eval()

# Tracking variables 
predictions , true_labels = [], []

# Predict 
for batch in tqdm(prediction_dataloader):
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels = batch
  
  # Telling the model not to compute or store gradients, saving memory and 
  # speeding up prediction
  with torch.no_grad():
      # Forward pass, calculate logit predictions
      outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)

  logits = outputs[0]

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  
  # Store predictions and true labels
  predictions.append(np.argmax(logits, axis=1).flatten())
  true_labels.append(label_ids)

print('    DONE.')

Predicting labels for 10,150 test sentences...


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # Remove the CWD from sys.path while we load stuff.



    DONE.


In [0]:
predictions  = np.concatenate(predictions)
true_labels = np.concatenate(true_labels)

In [0]:
#test
from sklearn.metrics import f1_score
print('f1-micro: ', f1_score(true_labels, predictions, average='micro')*100)
print('f1-weighted: ', f1_score(true_labels, predictions, average='weighted')*100) 

f1-micro:  74.46305418719213
f1-weighted:  74.35299219628398


In [0]:
#val
from sklearn.metrics import f1_score
print('f1-micro: ', f1_score(true_labels, predictions, average='micro')*100)
print('f1-weighted: ', f1_score(true_labels, predictions, average='weighted')*100)

f1-micro:  81.2237503088199
f1-weighted:  81.20123944010427


In [0]:
#train
from sklearn.metrics import f1_score
print('f1-micro: ', f1_score(true_labels, predictions, average='micro')*100)
print('f1-weighted: ', f1_score(true_labels, predictions, average='weighted')*100)

f1-micro:  89.96729888816219
f1-weighted:  89.96249572847556


In [0]:
#test translated
from sklearn.metrics import f1_score
print('f1-micro: ', f1_score(true_labels, predictions, average='micro')*100)
print('f1-weighted: ', f1_score(true_labels, predictions, average='weighted')*100)

f1-micro:  82.65024630541872
f1-weighted:  82.67452751994799


In [0]:
from sklearn.metrics import classification_report
print(classification_report(true_labels, predictions))

              precision    recall  f1-score   support

           0       0.87      0.84      0.85      1325
           1       0.79      0.83      0.81      2166
           2       0.86      0.87      0.86       978
           3       0.75      0.90      0.81       173
           4       0.83      0.77      0.80       473
           5       0.84      0.85      0.84      1480
           6       0.78      0.80      0.79      1723
           7       0.87      0.81      0.84      1832

    accuracy                           0.83     10150
   macro avg       0.82      0.83      0.83     10150
weighted avg       0.83      0.83      0.83     10150



In [0]:
from sklearn.metrics import classification_report
print(classification_report(true_labels, predictions)) 

              precision    recall  f1-score   support

           0       0.72      0.84      0.78      1325
           1       0.68      0.74      0.71      2166
           2       0.82      0.72      0.77       978
           3       0.74      0.50      0.60       173
           4       0.67      0.79      0.73       473
           5       0.76      0.61      0.68      1480
           6       0.77      0.78      0.78      1723
           7       0.80      0.78      0.79      1832

    accuracy                           0.74     10150
   macro avg       0.75      0.72      0.73     10150
weighted avg       0.75      0.74      0.74     10150



In [0]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(true_labels, predictions, normalize=None)) 

[[1115   80   33    6   30   15   39    7]
 [ 149 1602   74   15   57  122   89   58]
 [  89  131  708    6   10    8    6   20]
 [  31   37    3   87    3    2    4    6]
 [  13   42    1    0  373   14   18   12]
 [  58  341    2    0   49  899   70   61]
 [  61   70   16    1   10   32 1352  181]
 [  23   65   30    2   23   89  178 1422]]


In [0]:
lens = []
for i, text in enumerate(test_texts):
    if y_test[i] == 3:
        lens.append(len(text.split()))


In [0]:
np.mean(lens)

107.77456647398844

In [0]:
test_labels[:10]

['G', 'H', 'A', 'B', 'E', 'E', 'G', 'H', 'H', 'F']

In [0]:
predictions[:10]

array([6, 0, 0, 1, 4, 0, 6, 1, 0, 0])

In [0]:
test_labels[:10]

['G', 'H', 'A', 'B', 'E', 'E', 'G', 'H', 'H', 'F']

In [0]:
len(test_texts[1].split())

88

In [0]:
test_texts[2]

'использование сельское хозяйство и биотехнология. сущность изобретения растения регенеранты получают путем посадки стерильных эксплантов на питательную среду для каллусообразования, субкультивирования и индукции растений регенератов. при этом в качестве питательной среды используют среду мурасиге скуга, содержащую макро и микроэлементы, сахарозу, инозит, а также агар агар в количестве мг л, тиамин мг л, пиридоксин, мг л и никотиновую кислоту, мг л. на этапе каллусообразования в среду дополнительно вводят, дихлорфеноксиуксусную кислоту, мг л и бензиламинопурин, мг л, а на этапе субкультивирования каллуса в среду вводят, мг л, дихлорфеноксиуксусной кислоты'